In [ ]:
!pip install --upgrade pip

In [ ]:
!pip install pattern

In [ ]:
!pip install https://github.com/clips/pattern/archive/development.zip

In [ ]:
!pip install pixiedust

In [5]:
# import pixiedust

from pattern.en import parse, pprint, parsetree, wordnet

from pattern.vector import Document, Model, TFIDF
from pattern.vector import stem, PORTER, LEMMA
from pattern.vector import SVM, RADIAL, gridsearch, kfoldcv, count
from pattern.vector import distance, COSINE, tfidf

# For Latent Semantic Analysis (LSA)
import numpy as np
from numpy.linalg import svd
from numpy import dot, diag


import gensim
from gensim.models import doc2vec, KeyedVectors
from gensim.models.doc2vec import Doc2Vec, LabeledSentence

import pyemd

import re
import math
import operator
from collections import Counter

c:\python37\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [136]:
qlines = list()
alines = list()
# corpus = list()
documents = list()
qaDocs = list()
# container = list()
container = dict()

In [137]:
def readAndCreateCorpus():
    with open("FAQs.txt") as f:
        for line in f:
            if(re.match(r"^[0-9]{1,}.", line)):
                question = re.sub(r"^[0-9]{1,}. (.*)", "\\1", line)
                qlines.append(question)
                pass
            elif(re.match(r"^[A].", line)):
                answer = re.sub(r"^[A]. (.*)", "\\1", line)
                alines.append(answer)
                pass
            pass
        pass
    
    for i in range(len(qlines)):
        qa = qlines[i] + alines[i]
        # corpus.append(qa)
        # documents.append({"Q" : qlines[i], "A" : alines[i]})
        documents.append(qa)
        pass
    pass

In [138]:
readAndCreateCorpus()

In [139]:
parsetree(documents[0], lemmata = True)

[Sentence('What/WP/O/O/what ways/NNS/B-NP/O/way do/VBP/B-VP/O/do you/PRP/B-NP/O/you support/VB/B-VP/O/support payment/NN/B-NP/O/payment ?/./O/O/?'),
 Sentence('You/PRP/B-NP/O/you can/MD/B-VP/O/can pay/VB/I-VP/O/pay using/VBG/I-VP/O/used a/DT/B-NP/O/a credit/NN/I-NP/O/credit or/CC/O/O/or debit/NN/B-NP/O/debit card/NN/I-NP/O/card ././O/O/.'),
 Sentence('You/PRP/B-NP/O/you can/MD/B-VP/O/can also/RB/I-VP/O/also use/VB/I-VP/O/use your/PRP$/B-NP/O/your GameLink/NN/I-NP/O/gamelink points/NNS/I-NP/O/point to/TO/B-VP/O/to pay/VB/I-VP/O/pay for/IN/B-PP/B-PNP/for items/NNS/B-NP/I-PNP/item ././O/O/.')]

In [140]:
def createContainerAlt():
    for i in range(len(documents)):
        qaDocs.append(Document(documents[i], stemmer = LEMMA, name = "DOC {}".format(i)))
        
        # pos = parse(documents[i], lemmata = True)
        parse_tree = parsetree(documents[i], lemmata = True)
        
        DOC_WORDS = list()
        
        word = None
        pos = None
        lemma = None
        stemmed = None
        wordnetted = dict()
        
        for sentence in parse_tree:
            for w in sentence:
                word = w.string
                pos = w.pos
                lemma = w.lemma
                stemmed = stem(word, stemmer = LEMMA)

                hypernyms = list()
                hyponyms = list()
                holonyms = list()
                meronyms = list()

                synsetList = wordnet.synsets(word)
                for synset in synsetList:
                    hypernyms.append(synset.hypernyms())
                    hyponyms.append(synset.hyponyms())
                    holonyms.append(synset.holonyms())
                    meronyms.append(synset.meronyms())
                    pass

                wordnetted = {"hypernyms" : hypernyms,
                 "hyponyms" : hyponyms,
                 "holonyms" : holonyms,
                 "meronyms" : meronyms}
                
                DOC_WORDS.append({word : {"pos" : pos, "stemmed" : stemmed, "lemma" : lemma, "wordneted" : wordnetted}})
                pass
            pass
        
        # container.append({"DOC {}".format(i) : {"word" : word, "pos" : pos, "stemmed" : stemmed, "lemma" : lemma, "deptree" : parse_tree, "wordneted" : wordnetted}})
        # container.append({"DOC {}".format(i) : {"deptree" : parse_tree, "DOC_WORDS" : DOC_WORDS}})
        container.update({"DOC {}".format(i) : {"deptree" : parse_tree, "DOC_WORDS" : DOC_WORDS}})
        pass
    pass

In [141]:
createContainerAlt()

In [142]:
container

{'DOC 0': {'deptree': [Sentence('What/WP/O/O/what ways/NNS/B-NP/O/way do/VBP/B-VP/O/do you/PRP/B-NP/O/you support/VB/B-VP/O/support payment/NN/B-NP/O/payment ?/./O/O/?'),
   Sentence('You/PRP/B-NP/O/you can/MD/B-VP/O/can pay/VB/I-VP/O/pay using/VBG/I-VP/O/used a/DT/B-NP/O/a credit/NN/I-NP/O/credit or/CC/O/O/or debit/NN/B-NP/O/debit card/NN/I-NP/O/card ././O/O/.'),
   Sentence('You/PRP/B-NP/O/you can/MD/B-VP/O/can also/RB/I-VP/O/also use/VB/I-VP/O/use your/PRP$/B-NP/O/your GameLink/NN/I-NP/O/gamelink points/NNS/I-NP/O/point to/TO/B-VP/O/to pay/VB/I-VP/O/pay for/IN/B-PP/B-PNP/for items/NNS/B-NP/I-PNP/item ././O/O/.')],
  'DOC_WORDS': [{'What': {'pos': 'WP',
     'stemmed': 'what',
     'lemma': 'what',
     'wordneted': {'hypernyms': [],
      'hyponyms': [],
      'holonyms': [],
      'meronyms': []}}},
   {'ways': {'pos': 'NNS',
     'stemmed': 'way',
     'lemma': 'way',
     'wordneted': {'hypernyms': [[Synset('structure.n.01')],
       [Synset('property.n.02')],
       [Synset('imp

In [17]:
def createInputContainerAlt(inputSentence):
    inputContainer = dict()
    # pos = parse(inputSentence, lemmata = True)
    parse_tree = parsetree(inputSentence, lemmata = True)
    
    DOC_WORDS = list()

    word = None
    pos = None
    lemma = None
    stemmed = None
    wordnetted = dict()

    for sentence in parse_tree:
        for w in sentence:
            
            # print(w.__dict__)
            # print(dir(w))
            
            word = w.string
            pos = w.pos
            lemma = w.lemma
            stemmed = stem(word, stemmer = LEMMA)
            
            """
            print("word : {}".format(word))
            print("pos : {}".format(pos))
            print("lemma : {}".format(lemma))
            print("stemmed : {}".format(stemmed))
            print("\n")
            """

            hypernyms = list()
            hyponyms = list()
            holonyms = list()
            meronyms = list()

            synsetList = wordnet.synsets(word)
            for synset in synsetList:
                hypernyms.append(synset.hypernyms())
                hyponyms.append(synset.hyponyms())
                holonyms.append(synset.holonyms())
                meronyms.append(synset.meronyms())
                pass

            wordnetted = {"hypernyms" : hypernyms,
             "hyponyms" : hyponyms,
             "holonyms" : holonyms,
             "meronyms" : meronyms}
            
            DOC_WORDS.append({word : {"pos" : pos, "stemmed" : stemmed, "lemma" : lemma, "wordneted" : wordnetted}})
            pass
        pass
    
    inputContainer.update({"Input" : {"deptree" : parse_tree, "DOC_WORDS" : DOC_WORDS}})
    
    return inputContainer
    pass

In [18]:
def toList(part, wordList):
    parts = list()
    
    for words in wordList:
        for w in words:
            parts.append((w, words[w][part]))
            pass
        pass
    
    return parts
    pass

In [19]:
toList("pos", container["DOC 1"]["DOC_WORDS"])

[('Can', 'MD'),
 ('I', 'PRP'),
 ('pre-order', 'VBP'),
 ('games', 'NNS'),
 ('on', 'IN'),
 ('the', 'DT'),
 ('site', 'NN'),
 ('?', '.'),
 ('Yes', 'UH'),
 (',', ','),
 ('you', 'PRP'),
 ('can', 'MD'),
 ('definitely', 'RB'),
 ('pre-order', 'JJ'),
 ('games', 'NNS'),
 ('on', 'IN'),
 ('our', 'PRP$'),
 ('site', 'NN'),
 ('.', '.'),
 ('Some', 'DT'),
 ('games', 'NNS'),
 ('also', 'RB'),
 ('offer', 'VBP'),
 ('pre-order', 'NN'),
 ('bonuses', 'NNS'),
 ('and', 'CC'),
 ('discounts', 'NNS'),
 ('.', '.')]

In [20]:
toList("pos", inputContainer["Input"]["DOC_WORDS"])

NameError: name 'inputContainer' is not defined

In [21]:
Counter(toList("pos", inputContainer["Input"]["DOC_WORDS"]))

NameError: name 'inputContainer' is not defined

In [22]:
toList("wordneted", container["DOC 1"]["DOC_WORDS"])

[('Can',
  {'hypernyms': [[Synset('container.n.01')],
    [Synset('containerful.n.01')],
    [Synset('buoy.n.01')],
    [Synset('body_part.n.01')],
    [Synset('plumbing_fixture.n.01')],
    [Synset('room.n.01')]],
   'hyponyms': [[Synset('beer_can.n.01'),
     Synset('caddy.n.01'),
     Synset('cannikin.n.02'),
     Synset('coffee_can.n.01'),
     Synset('milk_can.n.01'),
     Synset('oilcan.n.01'),
     Synset('soda_can.n.01')],
    [],
    [],
    [],
    [Synset('flush_toilet.n.01'),
     Synset('flushless_toilet.n.01'),
     Synset('potty_seat.n.01')],
    [Synset('head.n.30'),
     Synset('public_toilet.n.01'),
     Synset('washroom.n.01'),
     Synset('water_closet.n.01')]],
   'holonyms': [[],
    [],
    [],
    [Synset('torso.n.01')],
    [Synset('bathroom.n.01'), Synset('toilet.n.01')],
    []],
   'meronyms': [[],
    [],
    [],
    [],
    [Synset('toilet_bowl.n.01'), Synset('toilet_seat.n.01')],
    [Synset('toilet.n.02')]]}),
 ('I',
  {'hypernyms': [[Synset('chemical_el

In [23]:
conParts = toList("wordneted", container["DOC 1"]["DOC_WORDS"])
conParts
# [conPart[1] for conPart in conParts]

[('Can',
  {'hypernyms': [[Synset('container.n.01')],
    [Synset('containerful.n.01')],
    [Synset('buoy.n.01')],
    [Synset('body_part.n.01')],
    [Synset('plumbing_fixture.n.01')],
    [Synset('room.n.01')]],
   'hyponyms': [[Synset('beer_can.n.01'),
     Synset('caddy.n.01'),
     Synset('cannikin.n.02'),
     Synset('coffee_can.n.01'),
     Synset('milk_can.n.01'),
     Synset('oilcan.n.01'),
     Synset('soda_can.n.01')],
    [],
    [],
    [],
    [Synset('flush_toilet.n.01'),
     Synset('flushless_toilet.n.01'),
     Synset('potty_seat.n.01')],
    [Synset('head.n.30'),
     Synset('public_toilet.n.01'),
     Synset('washroom.n.01'),
     Synset('water_closet.n.01')]],
   'holonyms': [[],
    [],
    [],
    [Synset('torso.n.01')],
    [Synset('bathroom.n.01'), Synset('toilet.n.01')],
    []],
   'meronyms': [[],
    [],
    [],
    [],
    [Synset('toilet_bowl.n.01'), Synset('toilet_seat.n.01')],
    [Synset('toilet.n.02')]]}),
 ('I',
  {'hypernyms': [[Synset('chemical_el

In [24]:
def getSimilarity(part, containerDict, inputDict):
    
    """
    print(part)
    print("\n")
    print(containerDict["DOC_WORDS"])
    print("\n")
    print(inputDict["DOC_WORDS"])
    print("\n\n")
    """
    
    similarity = 0
    
    conParts = toList(part, containerDict["DOC_WORDS"])
    inParts = toList(part, inputDict["DOC_WORDS"])
    
    # conCountPart = [conPart[1] for conPart in conParts]
    # inCountPart = [inPart[1] for inPart in inParts]
    
    
    # Calculate Cosine Similarity
    def get_cosine(vec1, vec2):
        intersection = set(vec1.keys()) & set(vec2.keys())
        numerator = sum([vec1[x] * vec2[x] for x in intersection])
        
        sum1 = sum([vec1[x]**2 for x in vec1.keys()])
        sum2 = sum([vec2[x]**2 for x in vec2.keys()])
        denominator = math.sqrt(sum1) * math.sqrt(sum2)
        
        if not denominator:
            return 0.0
            pass
        else:
            return float(numerator) / denominator
            pass
        pass
        
    
    if (part == "wordneted"):
        
        hyperSim = hypoSim = holoSim = meroSim = 0
        
        wordnetParts = ["hypernyms", "hyponyms", "holonyms", "meronyms"]
        
        # print(conCountPart)
        print(inCountPart)
        
        conCounter = Counter(conCountPart)
        # inCounter = Counter(inCountPart)
        
        # print(get_cosine(conCounter, inCounter))
        
        pass
    else:
        # conCounter = Counter(conCountPart)
        # inCounter = Counter(inCountPart)
        
        conCounter = Counter(conParts)
        inCounter = Counter(inParts)

        """
        print(conCounter)
        print("\n")
        print(inCounter)
        print("\n\n")
        """
        
        similarity = get_cosine(conCounter, inCounter)
        pass
    
    # similarity = distance(conCounter, inCounter, method=COSINE)
    return similarity
    pass

In [25]:
getSimilarity("pos", container["DOC 49"], inputContainer["Input"])

NameError: name 'inputContainer' is not defined

In [26]:
inputAlt1 = createInputContainerAlt("What ways do you support payment?")
inputAlt2 = createInputContainerAlt("What ways do you support payment?")
getSimilarity("pos", inputAlt1["Input"], inputAlt2["Input"])

0.9999999999999999

In [27]:
getSimilarity("pos", container["DOC 49"], inputContainer["Input"])

NameError: name 'inputContainer' is not defined

In [ ]:
def testSimilarity():
    availableParts = ["pos", "stemmed", "lemma", "wordneted"]
    
    for availablePart in availableParts:
        print("TESTING FOR {}".format(availablePart))
        
        simList = list()
        for doc in container:
            similarity = getSimilarity(availablePart, container[doc], inputContainer["Input"])
            simList.append((doc, similarity))
            pass
        print(sorted(simList, key = lambda x : x[1], reverse = True))
        # simList.clear()
        del simList
        
        print("\n\n\n")
        pass
    pass

In [ ]:
testSimilarity()

In [ ]:
in1 = createInputContainerAlt("What ways do you support payment?")
# in2 = createInputContainerAlt("Do you support online payment?")
in2 = createInputContainerAlt("Do you support online payment?")

in1 = createInputContainerAlt("What ways do you support payment?")
in2 = createInputContainerAlt("Is this a different support sentence?")

In [ ]:
getSimilarity("pos", in1["Input"], in2["Input"])

In [ ]:
getSimilarity("lemma", in1["Input"], in2["Input"])

In [ ]:
from pattern.vector import COSINE, EUCLIDEAN, MANHATTAN, HAMMING
import math
q = {('What', 'WP'): 1, ('ways', 'NNS'): 1, ('do', 'VBP'): 1, ('you', 'PRP'): 1, ('support', 'VB'): 1, ('payment', 'NN'): 1, ('?', '.'): 1}

def get_cosine(vec1, vec2):
     intersection = set(vec1.keys()) & set(vec2.keys())
     numerator = sum([vec1[x] * vec2[x] for x in intersection])

     sum1 = sum([vec1[x]**2 for x in vec1.keys()])
     sum2 = sum([vec2[x]**2 for x in vec2.keys()])
     denominator = math.sqrt(sum1) * math.sqrt(sum2)

     if not denominator:
        return 0.0
     else:
        return float(numerator) / denominator

get_cosine(q, q)

In [ ]:
document.vector

In [ ]:
bagofwords = [Counter(re.findall(r'\w+', txt)) for txt in documents]

In [ ]:
# Calculating the match score for the input query against each of the FAQs

def match_qa_pair(input_bag):
    match_score = dict()
    for word in input_bag:
        for w in word:
            for i in range(len(bagofwords)):
                if( w in bagofwords[i].keys()):
                    match_score[i] = match_score.get(i, 0) + 1
                    pass
                else:
                    match_score[i] = 0
                    pass
                pass
            pass
        pass
    
    #print(match_score)
    maximum = max(match_score.items(), key=lambda k : k[1])
    #print(maximum[0])
    for key in match_score:
        if key == maximum[0]:
            print("The matching question is: " + qlines[key])
            print("The matching answer is: " + alines[key])
            pass
        pass
    pass


In [ ]:
document = Document(sentence, stemmer = LEMMA)
pos = parse(sentence, lemmata = True)
parse_tree = parsetree(sentence, lemmata = True)

In [ ]:
faqDoc = Document(" ".join(corpus))

In [ ]:
len(set(faqDoc.vector))

In [ ]:
# pprint(parse(sentence, relations=True, lemmata=True))
# pprint(parsetree(sentence, relations=True, lemmata=True))

In [ ]:
for word in sentence.split():
    synsetList = wordnet.synsets(word)
    
    print(word)
    for synset in synsetList:
        print("Definition : {}".format(synset.gloss))
        print("Synonyms : {}".format(synset.synonyms))
        print("Hypernyms : {}".format(synset.hypernyms()))
        print("Hyponyms : {}".format(synset.hyponyms()))
        print("Holonyms : {}".format(synset.holonyms()))
        print("Meronyms : {}".format(synset.meronyms()))
        pass
    
    print("\n")
    pass

In [100]:
model = KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True)

In [162]:
def prettyContainerDoc(qa):
    pprint(qa.get("deptree"))
    
    for doc in qa["DOC_WORDS"]:
        # print(doc.keys())
        for d in doc.keys():
            # print("""
            # {:-^13} : {}
            # {:-^13} : {}
            # {:-^13} : {}
            # {:-^13} : {}
            # """.format("Word", d,
              #       "POS Tags", doc[d]["pos"],
               #      "Stem", doc[d]["stemmed"],
                #     "Lemma", doc[d]["lemma"]))
                
            print("""
            {:-^13} : {}
            {:-^13} : {}
            """.format("Word", d,
                     "Stem", doc[d]["stemmed"]))
            
            print("""
            {:-^13} : {}
            {:-^13} : {}
            {:-^13} : {}
            {:-^13} : {}
            """.format("Hypernyms", doc[d]["wordneted"]["hypernyms"],
                       "Hyponyms", doc[d]["wordneted"]["hyponyms"],
                       "Holonyms", doc[d]["wordneted"]["holonyms"],
                       "Meronyms", doc[d]["wordneted"]["meronyms"]))

            pass
        pass
    
    
    print("""{:-^13} : {}""".format("WM Distance", qa.get("WM Distance")))
    
    pass

In [163]:
for qa in container:
    prettyContainerDoc(container[qa])
    pass

          WORD   TAG    CHUNK   ROLE   ID     PNP    LEMMA     
                                                               
          What   WP     -       -      -      -      what      
          ways   NNS    NP      -      -      -      way       
            do   VBP    VP      -      -      -      do        
           you   PRP    NP      -      -      -      you       
       support   VB     VP      -      -      -      support   
       payment   NN     NP      -      -      -      payment   
             ?   .      -       -      -      -      ?         

          WORD   TAG    CHUNK   ROLE   ID     PNP    LEMMA    
                                                              
           You   PRP    NP      -      -      -      you      
           can   MD     VP      -      -      -      can      
           pay   VB     VP ^    -      -      -      pay      
         using   VBG    VP ^    -      -      -      used     
             a   DT     NP      -      -     

            

            ----Word----- : language
            ----Stem----- : language
            

            --Hypernyms-- : [[Synset('communication.n.02')], [Synset('auditory_communication.n.01')], [Synset('text.n.01')], [Synset('higher_cognitive_process.n.01')], [Synset('faculty.n.01')], [Synset('word.n.01')]]
            --Hyponyms--- : [[Synset('artificial_language.n.01'), Synset('barrage.n.01'), Synset('dead_language.n.01'), Synset('indigenous_language.n.01'), Synset('lingua_franca.n.01'), Synset('metalanguage.n.01'), Synset('native_language.n.01'), Synset('natural_language.n.01'), Synset('object_language.n.02'), Synset('sign_language.n.01'), Synset('slanguage.n.01'), Synset('source_language.n.01'), Synset('string_of_words.n.01'), Synset('superstrate.n.02'), Synset('usage.n.03'), Synset('words.n.03')], [Synset('conversation.n.01'), Synset('dictation.n.02'), Synset('discussion.n.02'), Synset('idiolect.n.01'), Synset('monologue.n.02'), Synset('non-standard_speech.n.01'), Synset

In [104]:
wmDistDict = dict()

punctuation = " .,;:!?()[]{}`''\"@#$^&*+-|=~_"

# inSentence = "Can I play games when I am offline?"
inSentence = "Can I play games without an internet connection?"

for doc in range(len(documents)):
    distance = model.wmdistance(documents[doc], inSentence)
    wmDistDict.update({"DOC {}".format(doc) : distance})
    pass

# print('distance = %.3f' % distance)

sorted(wmDistDict.items(), key = operator.itemgetter(1))

[('DOC 29', 0.636801587639892),
 ('DOC 33', 0.6484112102446657),
 ('DOC 32', 0.700338313028651),
 ('DOC 17', 0.7094318948700811),
 ('DOC 24', 0.7214693819741572),
 ('DOC 3', 0.7265633982238386),
 ('DOC 21', 0.726610938792867),
 ('DOC 18', 0.7431567866705039),
 ('DOC 28', 0.7456626291260415),
 ('DOC 35', 0.7540291047666804),
 ('DOC 31', 0.7589490421841338),
 ('DOC 43', 0.8022962191931773),
 ('DOC 20', 0.8380498626666297),
 ('DOC 30', 0.8405497406248917),
 ('DOC 14', 0.8410076669226992),
 ('DOC 23', 0.8464874744780073),
 ('DOC 49', 0.8471870032749217),
 ('DOC 15', 0.8503467025866162),
 ('DOC 13', 0.8680847412889308),
 ('DOC 12', 0.8687234731716837),
 ('DOC 2', 0.8751037379999697),
 ('DOC 19', 0.8883359939603518),
 ('DOC 34', 0.8888506629783768),
 ('DOC 50', 0.9267286211362606),
 ('DOC 16', 0.9292872765402916),
 ('DOC 53', 0.9339536691263269),
 ('DOC 45', 0.9367202836231181),
 ('DOC 22', 0.9754509026657285),
 ('DOC 47', 0.994248938633232),
 ('DOC 54', 0.9960188356347582),
 ('DOC 38', 0.99

In [147]:
def calcWMDistance(inSentence):
    wmDistDict = dict()
    
    for doc in range(len(documents)):
        distance = model.wmdistance(documents[doc], inSentence)
        wmDistDict.update({"DOC {}".format(doc) : distance})
        
        container["DOC {}".format(doc)].update({"WM Distance" : distance})
        pass
    
    return wmDistDict
    pass

In [148]:
wmDistance = calcWMDistance(inSentence)
sorted(wmDistance.items(), key = operator.itemgetter(1))

[('DOC 29', 0.636801587639892),
 ('DOC 33', 0.6484112102446657),
 ('DOC 32', 0.700338313028651),
 ('DOC 17', 0.7094318948700811),
 ('DOC 24', 0.7214693819741572),
 ('DOC 3', 0.7265633982238386),
 ('DOC 21', 0.726610938792867),
 ('DOC 18', 0.7431567866705039),
 ('DOC 28', 0.7456626291260415),
 ('DOC 35', 0.7540291047666804),
 ('DOC 31', 0.7589490421841338),
 ('DOC 43', 0.8022962191931773),
 ('DOC 20', 0.8380498626666297),
 ('DOC 30', 0.8405497406248917),
 ('DOC 14', 0.8410076669226992),
 ('DOC 23', 0.8464874744780073),
 ('DOC 49', 0.8471870032749217),
 ('DOC 15', 0.8503467025866162),
 ('DOC 13', 0.8680847412889308),
 ('DOC 12', 0.8687234731716837),
 ('DOC 2', 0.8751037379999697),
 ('DOC 19', 0.8883359939603518),
 ('DOC 34', 0.8888506629783768),
 ('DOC 50', 0.9267286211362606),
 ('DOC 16', 0.9292872765402916),
 ('DOC 53', 0.9339536691263269),
 ('DOC 45', 0.9367202836231181),
 ('DOC 22', 0.9754509026657285),
 ('DOC 47', 0.994248938633232),
 ('DOC 54', 0.9960188356347582),
 ('DOC 38', 0.99

In [145]:
container["DOC 0"]

{'deptree': [Sentence('What/WP/O/O/what ways/NNS/B-NP/O/way do/VBP/B-VP/O/do you/PRP/B-NP/O/you support/VB/B-VP/O/support payment/NN/B-NP/O/payment ?/./O/O/?'),
  Sentence('You/PRP/B-NP/O/you can/MD/B-VP/O/can pay/VB/I-VP/O/pay using/VBG/I-VP/O/used a/DT/B-NP/O/a credit/NN/I-NP/O/credit or/CC/O/O/or debit/NN/B-NP/O/debit card/NN/I-NP/O/card ././O/O/.'),
  Sentence('You/PRP/B-NP/O/you can/MD/B-VP/O/can also/RB/I-VP/O/also use/VB/I-VP/O/use your/PRP$/B-NP/O/your GameLink/NN/I-NP/O/gamelink points/NNS/I-NP/O/point to/TO/B-VP/O/to pay/VB/I-VP/O/pay for/IN/B-PP/B-PNP/for items/NNS/B-NP/I-PNP/item ././O/O/.')],
 'DOC_WORDS': [{'What': {'pos': 'WP',
    'stemmed': 'what',
    'lemma': 'what',
    'wordneted': {'hypernyms': [],
     'hyponyms': [],
     'holonyms': [],
     'meronyms': []}}},
  {'ways': {'pos': 'NNS',
    'stemmed': 'way',
    'lemma': 'way',
    'wordneted': {'hypernyms': [[Synset('structure.n.01')],
      [Synset('property.n.02')],
      [Synset('implementation.n.02')],
    

In [146]:
container

{'DOC 0': {'deptree': [Sentence('What/WP/O/O/what ways/NNS/B-NP/O/way do/VBP/B-VP/O/do you/PRP/B-NP/O/you support/VB/B-VP/O/support payment/NN/B-NP/O/payment ?/./O/O/?'),
   Sentence('You/PRP/B-NP/O/you can/MD/B-VP/O/can pay/VB/I-VP/O/pay using/VBG/I-VP/O/used a/DT/B-NP/O/a credit/NN/I-NP/O/credit or/CC/O/O/or debit/NN/B-NP/O/debit card/NN/I-NP/O/card ././O/O/.'),
   Sentence('You/PRP/B-NP/O/you can/MD/B-VP/O/can also/RB/I-VP/O/also use/VB/I-VP/O/use your/PRP$/B-NP/O/your GameLink/NN/I-NP/O/gamelink points/NNS/I-NP/O/point to/TO/B-VP/O/to pay/VB/I-VP/O/pay for/IN/B-PP/B-PNP/for items/NNS/B-NP/I-PNP/item ././O/O/.')],
  'DOC_WORDS': [{'What': {'pos': 'WP',
     'stemmed': 'what',
     'lemma': 'what',
     'wordneted': {'hypernyms': [],
      'hyponyms': [],
      'holonyms': [],
      'meronyms': []}}},
   {'ways': {'pos': 'NNS',
     'stemmed': 'way',
     'lemma': 'way',
     'wordneted': {'hypernyms': [[Synset('structure.n.01')],
       [Synset('property.n.02')],
       [Synset('imp